In [ ]:
import sys
sys.path.append('../')
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import sqlalchemy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib
from RtdRay import RtdRay
from mpl_toolkits.basemap import Basemap
from helpers.StationPhillip import StationPhillip

rtd = RtdRay(notebook=True)

### Delay analysis

In [ ]:
def get_delays(rtd_df):
    rtd_df['ar_cancellations'] = rtd_df['ar_cs'] == 'c'
    rtd_df['ar_cancellation_time_delta'] = (rtd_df['ar_clt'] - rtd_df['ar_pt']) / pd.Timedelta(minutes=1)
    rtd_df['ar_delay'] = (rtd_df['ar_ct'] - rtd_df['ar_pt']) / pd.Timedelta(minutes=1)
    ar_mask = ((rtd_df['ar_cs'] != 'c')) & (rtd_df['ar_delay'].notnull())
    rtd_df['ar_on_time_3'] = rtd_df.loc[ar_mask, 'ar_delay'] < 4
    rtd_df['ar_on_time_5'] = rtd_df.loc[ar_mask, 'ar_delay'] < 6
    rtd_df['ar_fern_on_time_5'] = rtd_df.loc[rtd_df['f'] == 'F', 'ar_on_time_5']

    rtd_df['dp_cancellations'] = rtd_df['dp_cs'] == 'c'
    rtd_df['dp_cancellation_time_delta'] = (rtd_df['dp_clt'] - rtd_df['dp_pt']) / pd.Timedelta(minutes=1)
    rtd_df['dp_delay'] = (rtd_df['dp_ct'] - rtd_df['dp_pt']) / pd.Timedelta(minutes=1)
    dp_mask = ((rtd_df['dp_cs'] != 'c')) & (rtd_df['dp_delay'].notnull())
    rtd_df['dp_on_time_3'] = rtd_df.loc[dp_mask, 'dp_delay'] < 4
    rtd_df['dp_on_time_5'] = rtd_df.loc[dp_mask, 'dp_delay'] < 6
    rtd_df['dp_fern_on_time_5'] = rtd_df.loc[rtd_df['f'] == 'F', 'dp_on_time_5']

    return rtd_df

### Analysis per station 

In [ ]:
class PerStationAnalysis(StationPhillip):
    FERN_ON_TIME_PLOT = {
        'count_1': 'ar_fern_on_time_5',
        'count_2': 'dp_fern_on_time_5',
        'color_value': 'dp_fern_on_time_5'
    }

    ALL_ON_TIME_PLOT = {
        'count_1': 'ar_delay',
        'count_2': 'dp_delay',
        'color_value': 'dp_on_time_5'
    }

    ALL_CANCELLATIONS_PLOT =  {
        'count_1': 'ar_delay',
        'count_2': 'dp_delay',
        'color_value': 'dp_cancellations'
    }

    def __init__(self, rtd_df, data=None):
        super().__init__(notebook=True)
        if data is not None:
            self.data = data
        else:
            self.data = rtd_df.groupby('station').agg({
                    'ar_delay': ['mean', 'count'],
                    'ar_on_time_3': ['mean'],
                    'ar_on_time_5': ['mean'],
                    'ar_cancellations': ['mean'],
                    'ar_cancellation_time_delta': ['mean', 'count'],
                    'ar_fern_on_time_5': ['mean', 'count'],
                    'dp_delay': ['mean', 'count'],
                    'dp_on_time_3': ['mean'],
                    'dp_on_time_5': ['mean'],
                    'dp_cancellations': ['mean'],
                    'dp_cancellation_time_delta': ['mean', 'count'],
                    'dp_fern_on_time_5': ['mean', 'count']
                }).compute()
            # remove station with less than 500 stops
            self.data = self.data.loc[self.data[('dp_delay', 'count')] > 500, :]

    def plot(self, data_to_plot):
        left = 5.67
        right = 15.64
        bot = 47.06
        top = 55.06
        plt.figure(figsize=(90/2,50/2))
        m = Basemap(llcrnrlon=left,llcrnrlat=bot,urcrnrlon=right,urcrnrlat=top,
                    resolution='i', projection='tmerc', lat_0 = 51, lon_0 = 10)
        m.drawcoastlines(linewidth=0.72, color='black')
        m.drawcountries(zorder=0, color='black')

        x = np.zeros(len(self.data.index))
        y = np.zeros(len(self.data.index))
        s = np.zeros(len(self.data.index))
        c = np.zeros(len(self.data.index))

        for i, station in enumerate(self.data.index):
            x[i], y[i] = self.get_location(name=station)
            s[i] = (self.data.loc[station, [(data_to_plot['count_1'], 'count')]][0] +
                self.data.loc[station, [(data_to_plot['count_2'], 'count')]][0])
            c[i] = self.data.loc[station, [(data_to_plot['color_value'], 'mean')]]

        c = (c - min(c)) / max(c - min(c))
        self.c = c

        cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red", 'yellow',"green"])
        m.scatter(x, y, c=c, cmap=cmap, s=s, alpha=0.2, latlon=True)

In [ ]:
rtd_df = rtd.load_data(columns=['station', 'c', 'ar_ct', 'ar_pt', 'dp_ct', 'dp_pt', 'ar_cs', 'ar_clt', 'dp_cs', 'dp_clt', 'f'])

In [ ]:
rtd_df = get_delays(rtd_df)

In [ ]:
per_station = PerStationAnalysis(rtd_df)

In [ ]:
per_station = PerStationAnalysis(rtd_df, per_station.data)

In [ ]:
per_station.plot(per_station.ALL_ON_TIME_PLOT)
plt.show()

### Datapoints over time

In [ ]:
rtd_df = rtd.load_data(columns=['ar_pt'])

In [ ]:
rtd_df['ar_pt'] = rtd_df['ar_pt'].dt.round(freq='D')

In [ ]:
over_time = rtd_df.groupby('ar_pt').agg({'ar_pt': ['count']}).compute()

In [ ]:
over_time.plot(kind='area')

### train type stuff

In [ ]:
def train_types_bubble_chart(rtd_df):
    train_types = rtd_df['c']
    per_train_type = train_types.value_counts()
    per_train_type = np.sqrt(per_train_type.to_numpy() / np.pi)
    print(per_train_type)
    bubble_plot = BubbleChart(per_train_type)

    fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))
    ax.axis("off")

    bubble_plot.collapse()

    bubble_plot.plot(ax)
    ax.relim()
    ax.autoscale_view()
    plt.show()

In [ ]:
def train_types_pie_chart(rtd_df):
    train_types = rtd_df['c']
    per_train_type = train_types.value_counts()
    per_train_type
    per_train_type.plot.pie(figsize=(30, 30))

### other

In [ ]:
def data_over_time(rtd_df):
    rtd_df['arr'].plot()

In [ ]:
def trains_per_station_map_bubble_plot(rtd_df):
    trains_per_station = rtd_df['station'].value_counts()


In [ ]:
def delay_over_time(delays):
    delays['arr_delay'] = delays['arr_changed_time'] - delays['arr']
    delays['arr_unix'] = delays['arr'] / 60
    delays['arr'] = dd.to_datetime(delays['arr'] - delays['arr'] % (60 * 60 * 24), unit='s')
    delays = delays.groupby('arr').agg({'arr_unix':['std'], 'arr_delay': ['mean', 'std', 'max', 'min']})
    with ProgressBar():
        return delays.compute()

In [ ]:
train_types = rtd.load_data(columns=['c']).compute()
train_types_pie_chart(train_types)

In [ ]:
train_types_pie_chart(train_types)

In [ ]:
train_types_bubble_chart(train_types)

In [ ]:
rtd = RtdRay(notebook=True)
delays = rtd.load_data(columns=['arr', 'arr_changed_time'])
grouped_by = delay_over_time(delays)

In [ ]:
plot = grouped_by.reset_index().plot.scatter(y=('arr_delay', 'max'), x=('arr'))

In [ ]:
grouped_by.reset_index()

In [ ]:
rtd.load_data(columns=['arr', 'arr_changed_time'])['arr_changed_time'].head()

In [ ]:
delays = rtd.load_data(columns=['arr', 'arr_changed_time', 'arr_changed_platform', 'arr_changed_status', 'stay_time'])
delays_df = delays.compute()
delays_df['arr_changed_time'] - delays_df['arr']

In [ ]:
rtd['ar_pt'] = rtd['ar_pt'].astype('Int64').astype('str')
rtd['ar_ct'] = rtd['ar_ct'].astype('Int64').astype('str')

In [ ]:
rtd['ar_pt'] = pd.to_datetime(rtd['ar_pt'], format='%y%m%d%H%M', errors='coerce')

In [ ]:
rtd['ar_ct'] = pd.to_datetime(rtd['ar_ct'], format='%y%m%d%H%M', errors='coerce')

In [ ]:
rtd['ar_delay'] = rtd['ar_ct'] - rtd['ar_pt']